### Data Mining II --- Final Project --- Jurgen Xhafaj
#### Spring 2018
### Using Data Science to get predictions on soccer matches

# PLEASE READ README.txt FILE PRIOR TO PROCEEDING.

####   Overview
   The project I chose is about the beautiful game of soccer. Especially in Europe, soccer has an immense popularity. Having lived in a society that is greatly inclined towards making money quick and without effort, I have been exposed to a phenomenon that seems to be a plague for the Albanian society: gambling.

   Even though I have only spent 21 dollars in "gambling" activities (20 dollars at Mohegan Sun in CT, and 1 dollar at a betting center in Tirana, Albania), I must say that I am rather familiar with how soccer bets work.
   
   The goals of this project are related to using pre-given betting odds to predict the outcome of the match. Such betting odds vary among betting companies, as they do not value teams in the same way as each other.

   Regarding the ethical aspects of this project, I do not promote gambling of any form. That said, using the betting odds data should not bring any moral or social prejudice upon the user, since said odds are ALWAYS read from someone who is about to write a soccer ticket with a list of matches and the result they think the match will have. Such a ticket is displayed below:
![title](ticket.jpg)

### Data Structure
   The main table I used is the file 'edited match table.csv'. It is a very condensed version of the original 'match.csv' table, which I got from a soccer database in Kaggle (courtesy of Hugo Mathien) and football-data.co.uk. 
   
   This table includes several columns such as league, home and away team api numbers, as well as betting odds coefficients from up to 10 betting companies. Below are just some of the features of this data:
    * +25,000 matches
    * 11 European Countries with their lead championship
    * Seasons 2008 to 2016
    * Team line up with squad formation (X, Y coordinates)
    * Betting odds from up to 10 providers
    * Detailed match events (goal types, possession, corner, cross, fouls, cards etc...) for +10,000 matches
    
 The data has been collected from the following leagues:
    * Belgium Jupiler League
    * England Premier League
    * France Ligue 1
    * Germany 1. Bundesliga
    * Italy Serie A
    * Netherlands Eredivisie
    * Poland Ekstraklasa
    * Portugal Liga ZON Sagres
    * Scotland Premier League
    * Spain LIGA BBVA
    * Switzerland Super League


### Visualizing locations of stadiums using Orange
  Not only can we see where these stadiums are located, but we can also customize how the shapes of objects will look, as well as their color and their size. All of this is made possibly by the GeoMap package in Orange.
  Below is a very simple diagram that also shows the location of GeoMap in Orange, provided that it has been installed.
  The data used for the stadiums' locations is from the file 'stadiums_20150302.csv'.
![title](orange.png)

### Zooming out (notice clustering) and zooming in the map
![title](stadiums_out.png)
![title](stadiums_in.png)

### Goals of our project
 * Predict whether total number of goals is 0-2 or 3+
 * Predict the outcome of match with 3 outputs (1-home_win X-tie 2-away_win)
 * Predict the outcome with 2 outputs (home team wins or not)
 * Predict whether a specific strategy is more likely to win a home (or away) game

### Previous work (quote from author Hugo Mathien)
   "The Holy Grail... is obviously to predict the outcome of the game. The  bookies use 3 classes (Home Win, Draw, Away Win). 
   They get it right about 53% of the time. This is also what I've achieved so far using my own SVM. Though it may sound high for such a random sport game, you've got to know that the home team wins about 46% of the time. So the base case (constantly predicting Home Win) has indeed 46% precision."

### 1. Number of goals
##### When adding the total of goals, we get a range between 0 and 12. 
total_goals=home_team_goals-away_team_goals

The data is represented in a histogram below:
![title](total_goals.png)

### 2.&3. Game outcome
##### As I expected the models' accuracy scores were much higher when the output only had two options: whether the home team wins or not.
 Note that when betting, the model predicting if home team wins or not would be the most efficient one, cost wise. You get ~65% accuracy on whether the home team wins or not, whereas the model with 3 outcomes only gives 36% accuracy. That said, these are not absolute figures, and caution should be exercised when dealing with teams like Arsenal, which regularly beats tough teams and gets schooled on a regular basis by small teams.

As you can see on the histogram below, the data is skewed to the right, which shows a tendency that a team wins more games when playing at home.
![title](game_outcome.png)

### 4. Finding strategies that win the most games (option1: home games, option2: away games)
##### Before I start, one should note that for an average team playing against another average team, getting a draw in an away game is a rather good result.
#### __Unfortunately__ , this objective could not be achieved for the following reasons:
 Initially, I was planning on this workflow:
     * First, only select categorical columns from the strategies table (called team_attributes.csv). This is because the numerical columns were statistics of the team's performance AFTER the match, and they would not be provided BEFORE the match (obviously).
     * Next, I would use one-hot encoding to transform the strings into arrays and then create new columns with the new information. The columns have 3 or 2 options, so I would get somewhere between 2 to 3 times as many columns. In this case, I would get 9$*$3 + 3$*$2 = 33 columns that would substitute 12 initial columns with strategy data.
     * Then, I would join two tables, one of which has the betting odds and the other containing strategy information. The joining points would be date columns and team ID's (called api's here).

#### And that is exactly where the problem was: one of the tables provided team_api's, and the other provided team_fifa_api's, which are two different numbers, unfortunately. If they had the same identifier label, then I would proceed with the models and Grid Search procedure. The issue is shows in the snapshot below:

P.S. It was very worrying to see that a team had two matches in a day, as shown on the snapshot below. It almost makes me think that the strategy data was collected in a rush.

![title](issue.png)

### Selecting the relevant data
   Analyzing the columns and checking which are going to help in making predictions
   Here, the betting coefficients columns contain data given prior to the match, so it is reasonable to include them.
   The team names have not been taken into account because certain teams win much more than others do.
   League code is not considered because certain leagues have overall much better/worse teams.
   Some matches have a lot of betting coefficients and some others do not. However, coefficients from Bet365 company were as good alone as combined with other predictions.
    Note that initially I had only betting coefficients as input data, but for some cases it was useful to include the stage of the match. 
    
    Stage = the game number (a team plays 2*(n-1) games where n is nr of teams in championship
Typically, a championship has 20 teams, so a team plays 2*(20-1)=38 games (labeled as stages here)

### Pre-processing the data
 * Prior to creating a Jupyter notebook, I stripped the match.csv document from an excessive number of useless columns, which were lacking data on >80% of the cases. The original table was over 200Mb, and 'edited match table.csv' is under 2 Mb.
 * New column: This varies according to the goal, but in this case it was the total number of goals in the match.
 * Creating np.array for our input and target columns, as well as a new dataframe that only contains the relevant data for the models. An example is given below:
 
     * Initial table
     ![title](tons of columns.png)
     * Simplified table with relevant data to predictions with 3 outcomes (1,2,3)
     ![title](1x2.png)
     
     
 * Separating the data into training and testing data. The ratio I chose was 80% for training and 20% for testing. Some classifiers had built-in k-fold cross validation so there was no need to test. Either way, the data includes more than 25000 matches, so the 20% is not a significant portion if left out completely, since it leaves 20000 matches, which is more than enough to train and perform 10 fold cross validation.
 
### Creating models 
 * I started with neural network, then proceeded with random forest and other classifiers, as shows on the table below.
 * The cross validation was always 10 fold cross validation, unless otherwise noted. Note that cases with 3 fold cross validation were simply made to get an idea of what to expect from 10 fold cross validation, and further work was done with the latter case.
 * OVERALL, the neural network worked the best when predicting the total number of goals in a game, whereas random forest worked best for predicting the actual outcome of the game (win, tie, loss for home team), and Voting Ensemble worked best when trying to predict if home team wins or not.
 ![title](voting1.png)
 Below, you can see the accuracy and time it took to run for Voting Ensemble.
 ![title](voting2.png)
 
### Performing Grid Search to tune parameters 
 * This step took a significant amount of time with neural networks, as expected
 * There were much more parameters to tune in neural networks, so I made sure to run multiple Grid searches on those cases.
 * I would not recommend that you run some of the neural network grid searches that I have here, since some of them take a lot of time. The longest one was the one that dealt with the optimizer. The accuracy gain was rather marginal but I let it finish for the sake of completeness.
 Note: the time it took to run is shown at the last line and the unit is seconds. In this case it took a whole Sopranos' episode to finish. 
 ![title](long.png)

## What would help our predictions?
##### Below are some suggestions as to what kind of data would be helpful in increasing our models' accuracy scores. Prior to stating them, I should say that I did find such data but it was for a very limited amount of matches and it lacked connecting labels with the main dataframe, so it was not too helpful. Here we go:
* Team's manager and their coaching record (Mourinho, Zidane, Guardiola, etc)
* Team's formation (5-4-1, 4-4-2, 4-3-3, etc)
* Weather conditions (some teams are favored and some are not)
* Players and their soccer games record (Take this with a grain of salt since the player condition can vary. Also, some key players' physical condition is usually 'injured', but once they recover, they perform exceptionally well in the field. Honorable mentions here are Neymar Jr and Gareth Bale.

Note: the truth is that some teams are notorious for rigging matches and should be avoided (especially Internazionale Milano, aka Inter), but that does require individual attention to every match. One way to correct this would be to just assume that our prediction accuracy should be 10% lower in Italian Seria A games and 5% lower in England's Premier League games.

## How do I feel after this project?
I think that the project helped me get a much better overall knowledge on a hands-on scenario which is definitely related to real life in my case. I feel much more confident with importing, manipulating data, building and validating models, as well as successfully performing Grid Search on several types of classifiers. One more thing is that I can now easily construct a neural network. As much as I'd like to build a CNN, I think that for this specific case we'd need models that would be time efficient and with low resource requirements, given that the best scenario with betting coefficients is when they are collected the day before/of the match, when they are the most accurate, money wise. I'm hoping that you're satisfied with my progress and that I will hopefully get an A in the course :) 